In [5]:
import pandas as pd
from sys import argv
from datetime import datetime
import os
#from tqdm.notebook import tqdm
#from tkinter.filedialog import askopenfilename

def logger(*args):
    """
        Print log message with excecuting file name, timestamp.
        *** All parameters should be on string-type!
    """
    try: __file__
    except NameError:
        message = f"[{str(datetime.now())}] {str(' '.join(args))}"
    else:
        message = f"[{__file__} {str(datetime.now())}] {str(' '.join(args))}"
    print(message)

logger("import & definition ends successfully.")

[2021-06-02 09:43:50.832928] import & definition ends successfully.


In [6]:
originPath = globals()['_dh'][0]
os.chdir(originPath)
#currentPath = os.getcwd()
targetFolderList = ['2020년자료', '2021년자료']
dataset = dict()
#dataset = {fName: {} for fName in targetFolderList}

for folderName in targetFolderList:
    targetDir = f"{originPath}/{folderName}"
    os.chdir(targetDir)
    logger(f'Working directory changed to {targetDir}')
    fileList = os.listdir(targetDir)
    for fileName in fileList:
        if fileName in dataset:
            dataset[fileName] = pd.concat([dataset[fileName], pd.read_csv(f"{targetDir}/{fileName}")], ignore_index=True)
        else:
            if fileName.endswith('.csv') and os.path.isfile(fileName):
                try:
                    df = pd.read_csv(fileName)
                    if len(df) > 0: dataset[fileName] = df
                    else: raise Exception
                except:
                    logger(f'Empty or wrong file [{fileName}] detected. jump to next file')
            else: continue
else:
    logger('Reading data from files complete.')

[2021-06-02 09:44:15.546640] Working directory changed to /Users/youngdae/Documents/미세먼지와 폭염 대응 그린쉘터 최적 이용망 분석 기술 개발3(국립산립과학원)/210526_측정망 DB 구축/측정넷자료/2020년자료
[2021-06-02 09:44:15.978435] Working directory changed to /Users/youngdae/Documents/미세먼지와 폭염 대응 그린쉘터 최적 이용망 분석 기술 개발3(국립산립과학원)/210526_측정망 DB 구축/측정넷자료/2021년자료
[2021-06-02 09:44:16.041913] Empty or wrong file [Nafion-태안_산단.csv] detected. jump to next file
[2021-06-02 09:44:16.472014] Empty or wrong file [BVOC-태안_산단.csv] detected. jump to next file
[2021-06-02 09:44:16.472571] Empty or wrong file [Met_태안_산단.csv] detected. jump to next file
[2021-06-02 09:44:16.498797] Empty or wrong file [AVOC-태안_산단.csv] detected. jump to next file
[2021-06-02 09:44:16.585258] Reading data from files complete.


In [7]:
columnSet = {'pm': ['stationName', 'measuredDatetime', 'PM10', 'PM25', 'PM1'],
             'weather': ['stationName', 'measuredDatetime', 'temperature', 'RelativeHumidity', 'windSpeed', 'windDirection']}

data_AVOC = pd.DataFrame(columns=columnSet['pm'])
data_BVOC = pd.DataFrame(columns=columnSet['pm'])
data_Nafion = pd.DataFrame(columns=columnSet['pm'])
data_weather = pd.DataFrame(columns=columnSet['weather'])

dSet = {'AVOC': {}, 'BVOC': {}, 'weather': {}, 'Nafion': {}}

# stationName 추가 및 컬럼명 변경
for key in dataset:
    if key.startswith('AVOC-') or key.startswith('BVOC-'):
        dataset[key].insert(0, "stationName", key[5:-4], True)
        dataset[key].columns = columnSet['pm']
        if key.startswith('AVOC-'): 
            dSet['AVOC'][key] = dataset[key]
        else:
            dSet['BVOC'][key] = dataset[key]
    elif key.startswith('Met_'):
        dataset[key].insert(0, "stationName", key[4:-4], True)
        dataset[key].columns = columnSet['weather']
        dSet['weather'][key] = dataset[key]
    elif key.startswith('Nafion-'):
        dataset[key].insert(0, "stationName", key[7:-4], True)
        dataset[key].columns = columnSet['pm']
        dSet['Nafion'][key] = dataset[key]
else:
    dataset, dSet = dSet, dataset
    logger('Data classification complete.')

for key in dataset:
    vars()['data_'+key] = pd.concat(list(dataset[key].values()), ignore_index=True)
else:
    logger('Data merged successfully.')

[2021-06-02 09:44:29.515305] Data classification complete.
[2021-06-02 09:44:29.605376] Data merged successfully.


In [8]:
os.chdir(originPath)

if os.path.isdir('nifos_data'):
    pass
else:
    os.makedirs('nifos_data')

for dataFile, fileName in zip((data_AVOC, data_BVOC, data_Nafion, data_weather), ['AVOC', 'BVOC', 'Nafion', 'weather']):
    dataFile.to_csv(f"{originPath}/nifos_data/{fileName}.csv", index=False)
else:
    logger('data export complete.')

[2021-06-02 09:44:39.658295] data export complete.


# Insert data into DB

In [24]:
import pymysql
import tkinter
from tkinter import filedialog
from sqlalchemy import create_engine
import keys

def getDframeName(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

originPath = globals()['_dh'][0]

filePicker = tkinter.Tk()
filePicker.withdraw()
dirPath = filedialog.askdirectory(parent=filePicker,initialdir=originPath,title='Please select a directory')
os.chdir(dirPath)
logger(f"Change working directory to {dirPath}")

fileList = [fileName for fileName in os.listdir() if fileName.endswith('.csv')]
dataset = dict()

for fileName in fileList:
    dataset[fileName] = pd.read_csv(fileName, encoding='utf-8')
else: 
    logger(f'{len(fileList)} file(s) read successfully.')
    display(dataset)

[2021-06-02 11:14:40.173557] Change working directory to /Users/youngdae/Documents/미세먼지와 폭염 대응 그린쉘터 최적 이용망 분석 기술 개발3(국립산립과학원)/210526_측정망 DB 구축/측정넷자료/nifos_data
[2021-06-02 11:14:40.829340] 4 file(s) read successfully.


{'BVOC.csv':        stationName     measuredDatetime       PM10       PM25        PM1
 0          양재_300m  2020-01-14 16:00:00  14.332667   4.704500   4.359500
 1          양재_300m  2020-01-14 17:00:00   6.797833   3.821167   3.584500
 2          양재_300m  2020-01-14 18:00:00   5.791500   3.744667   3.676167
 3          양재_300m  2020-01-14 19:00:00  10.353833   5.029167   4.564333
 4          양재_300m  2020-01-14 20:00:00   6.658167   4.197333   4.096000
 ...            ...                  ...        ...        ...        ...
 224685       광주_주거  2021-03-31 19:00:00  41.316667  20.500000  17.800000
 224686       광주_주거  2021-03-31 20:00:00  42.716667  24.750000  21.900000
 224687       광주_주거  2021-03-31 21:00:00  45.720000  27.050000  23.950000
 224688       광주_주거  2021-03-31 22:00:00  49.966667  29.083333  24.840000
 224689       광주_주거  2021-03-31 23:00:00  49.083333  33.883333  31.233333
 
 [224690 rows x 5 columns],
 'Nafion.csv':        stationName     measuredDatetime       PM10     

In [25]:
def connectDB():
    """
       Connect to MySQL database server, returning connection & cursor objects. 
    """
    try:
        connection = pymysql.connect(host=keys.HOST, port=keys.PORT, 
                                    user=keys.USERNAME, password=keys.PASSWORD, 
                                    database=keys.DBNAME)
        cursor = connection.cursor()
        logger('DB connection Established.')
        return connection, cursor
    except Exception as e:
        msg = f'DB connection failed! ERROR: {str(e)}'
        logger(msg)

try:
    connection = pymysql.connect()
    cursor = connection.cursor()
    logger('DB connection established.')
except Exception as e:
    logger(f'ERROR occured while connecting to DB server. \n EXCEPTION) {str(e)}')
    connection.close()

engine = create_engine(f'mysql+pymysql://{keys.USERNAME}:{keys.PASSWORD}@{keys.HOST}/{keys.DBNAME}')

for dataframe in dataset:
    dataset[dataframe].to_sql(name=dataframe, con=engine, if_exists='replace', index=False)
else: 
    connection.close()
    logger('Insert done!')

[2021-06-02 11:14:45.527984] DB connection established.
/Users/youngdae/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:1423: UserWarning: The provided table name 'BVOC.csv' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
/Users/youngdae/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:1423: UserWarning: The provided table name 'Nafion.csv' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
[2021-06-02 11:15:25.232145] Insert done!
/Users/youngdae/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:1423: UserWarning: The provided table name 'AVOC.csv' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warni